In [1]:
import jieba.analyse
import jieba
import codecs
from gensim import corpora
from gensim.summarization import bm25
import os
import re
import string
import pandas as pd

In [2]:
docid2context = {}
f = True
for line in open('NCPPolicies_context_20200301.csv'):
    if f: #跳过第一行
        f = False
        continue
    r = line.strip().split('\t')
    docid2context[r[0]] = r[1]

In [3]:
def tokenization(text):
    texts = []
    stoplist = {}.fromkeys([line.strip() for line in open('stopwords.txt',
                                                      'r', encoding='utf-8')])
    for item in text:
        segs = jieba.lcut(item)
        temp = [word for word in list(segs)
               if word.lstrip() is not None
               and word.lstrip() not in stoplist
               and word.lstrip() not in string.punctuation
               ]
        texts.append(temp)
    return texts

In [4]:
text = []
for item in docid2context:
    text.append(docid2context[item])

In [5]:
texts = tokenization(text)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\40821\AppData\Local\Temp\jieba.cache
Loading model cost 0.885 seconds.
Prefix dict has been built successfully.


In [6]:
dictionary = corpora.Dictionary(texts)
doc_vectors = [dictionary.doc2bow(text) for text in texts]
vec1 = doc_vectors[0]
vec1_sorted = sorted(vec1,key=lambda x:x[1],reverse=True)
print (len(vec1_sorted))
for term, freq in vec1_sorted[:5]:
    print (dictionary[term])

241
企业
疫情
裁员
一次性
参保


In [7]:
bm25Model = bm25.BM25(texts)
average_idf = sum(map(lambda k: float(bm25Model.idf[k]), bm25Model.idf.keys())) / len(bm25Model.idf.keys())

In [8]:
df = pd.read_csv('NCPPolicies_train_20200301.csv',sep = None)

C:\Users\40821\anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [9]:
question = df["question"]
questions = tokenization(question)

In [10]:
questions[0]

['工业', '信息化', '部到', '哪家', '企业', '进行', '督导', '检查']

In [11]:
scores = bm25Model.get_scores(questions[0])
scores.sort(reverse = True)
print(scores)

[19.190975880716266, 13.865447298456502, 13.851552151059181, 12.722471779278385, 12.409735902784085, 12.284681364970556, 12.22374729615204, 11.694291359658243, 11.639329893995662, 11.281056806024248, 11.248507388854957, 10.961043820852897, 10.948870745337413, 10.8443711124517, 10.830571331158149, 10.809704657407774, 10.790598477778685, 10.78553627482485, 10.744171044195896, 10.684856741576342, 10.488142553543321, 10.476297039289129, 10.328699698871624, 10.206132723440634, 10.182773990803732, 10.108047990501603, 10.073964614067295, 10.028771458425105, 9.987535618490462, 9.9567579917932, 9.891463620177703, 9.867629266182831, 9.837884131221717, 9.83049130196227, 9.755501407079096, 9.720579402028541, 9.715925827154809, 9.647581892949216, 9.607418345887503, 9.60504520021556, 9.512403519906307, 9.303366932295017, 9.23941552791499, 9.235203901840272, 9.199810660242246, 9.178007701441285, 9.177677783454287, 9.169036219628268, 9.164054375253698, 9.161093124917091, 9.158593066207745, 9.154437545

In [64]:
tflist = []
for item in questions:
    scores = bm25Model.get_scores(item)
    dict = {}
    for (item1, item2) in zip(docid2context,scores):
        dict[item1] = item2
    tf_idfSorted = sorted(dict.items(),key=lambda x:x[1],reverse=True)
    counter = 0
    QkeywordSorted = []
    for item in tf_idfSorted:
        QkeywordSorted.append(item[0])
        counter+=1
        if counter == 20:
            break
    tflist.append(QkeywordSorted)

In [13]:
length = len(df['docid'])
counter = 0
countg = 0
for (item1str, item2list) in zip(df['docid'],tflist):
    countg +=1
    if item1str in item2list:
        counter+=1
print('精度为:',counter/length)

精度为: 0.9134


In [20]:
import random

In [25]:
len(docid2context)

8932

In [10]:
order2id = {}
for item,item1 in zip(range(0,8932),docid2context):
    order2id[item] = item1

In [28]:
textstr = []
for item in texts:
    temp = ''
    for item1 in item:
        temp += ' '+item1
    textstr.append(temp)
textstr[0]

' 福建 部门 联合 出台 暖企 措施 支持 复 工稳 岗 解决 企业 复产 用工 困难 省政府 同意 省人 社厅 省工 信厅 省 教育厅 省 财政厅 省 交通运输 厅 省卫健委 联合 下发 通知 出台 一系列 暖企 措施 支持 疫情 防控 期间 复 工稳 岗 通知 明确 切实 发挥 各级 农民工 工作 领导小组 办公室 统筹 协调 作用 加强 劳务 用工 有效 对接 具备 外出 务工 条件 可成 规模 输送到 我省 工地 出行 前 14 天内 途 相关 症状 可由用 工地 输出地 联合 开展 点对点 一站式 直达 企业 专门 运输 省级 公共 就业 服务 机构 主要 劳务输出 省份 签订 劳务 协作 协议 设立 劳务 协作 工作站 每个 工作站 给予 一次性 10 万元 就业 服务 经费 补助 鼓励 优先 聘用 本地 劳务 人员 未经 省 应对 新冠 肺炎 疫情 工作 机构 确认 疫情 防控 急需 物资 生产 企业 引进 劳动力 一次性 用工 服务奖 补 标准 最高 提到 每人 2000 元 上述 企业 坚持 生产 一线 工作 职工 给予 每人每天 100 元 生活 补助 纳入 一次性 用工 服务奖 补 范畴 春节 当月 疫情 一级 响应 结束 月 采取 稳定 职工队伍 保持 连续 生产 企业 给予 一次性 稳 就业 奖补 加大 失业 稳岗 返还 力度 中小 微 企业 稳岗 返还 政策 裁员 率 标准 调整 高于 年度 全国 调查 失业率 控制目标 参保 职工 30 人 含 以下 企业 裁员 率 调整 超过 企业 参保 职工 总数 20% 裁员 或少 裁员 符合条件 参保 企业 返还 其上 年度 实际 缴纳 失业 保险费 50% 对受 疫情 影响 面临 暂时性 生产 经营 困难 恢复 有望 坚持 裁员 或少 裁员 符合条件 参保 企业 月 当地 月 人均 失业 保险金 参保 职工 人数 落实 失业 稳岗 返还 政策 加强 职业技能 培训 鼓励 技工 院校 学生 符合 疫情 防控 条件 下 参加 实习 实训 探索 简易 岗前 技能 培训 企业 生产 急需 新 录用 人员 每人 200 元 标准 一次性 给予 企业 简易 岗前 技能 培训 补贴 鼓励 实施 线上 培训 对受 疫情 影响 企业 停工 期 恢复期 组织 职工 参加 各类 线上 线下 职业培训 按规定 纳入

In [60]:
questionstr = []
for item in questions:
    temp = ''
    for item1 in item:
        temp += ' '+item1
    questionstr.append(temp)
questionstr[0]

' 工业 信息化 部到 哪家 企业 进行 督导 检查'

In [31]:
id2context = {}
for item,item1 in zip(docid2context, range(0,8932)):
    id2context[item] = item1

In [63]:
len(tflist) #要注意下面也是有tflist,并且长度不一样

1643

可以提升的地方是可以随机打乱数据，不过现在比较懒，没有使用这个方法

In [69]:
with open('test.txt','a') as file_handle:
    for item in range(4700, 5000):
        for item2 in tflist[item]:
            file_handle.write(questionstr[item])
            file_handle.write('\t')
            file_handle.write(textstr[id2context[item2]])
            file_handle.write('\t')
            file_handle.write('1')
            file_handle.write('\n')
            temp = random.randint(0, 8931)
            file_handle.write(questionstr[item])
            file_handle.write('\t')
            file_handle.write(textstr[temp])
            file_handle.write('\t')
            file_handle.write('0')
            file_handle.write('\n')
            temp = random.randint(0, 8931)
            file_handle.write(questionstr[item])
            file_handle.write('\t')
            file_handle.write(textstr[temp])
            file_handle.write('\t')
            file_handle.write('0')
            file_handle.write('\n')
            temp = random.randint(0, 8931)
            file_handle.write(questionstr[item])
            file_handle.write('\t')
            file_handle.write(textstr[temp])
            file_handle.write('\t')
            file_handle.write('0')
            file_handle.write('\n')
            temp = random.randint(0, 8931)
            file_handle.write(questionstr[item])
            file_handle.write('\t')
            file_handle.write(textstr[temp])
            file_handle.write('\t')
            file_handle.write('0')
            file_handle.write('\n')
            temp = random.randint(0, 8931)
            file_handle.write(questionstr[item])
            file_handle.write('\t')
            file_handle.write(textstr[temp])
            file_handle.write('\t')
            file_handle.write('0')
            file_handle.write('\n')
#         if item1==3499:
#             break

In [8]:
test = pd.read_csv(open("NCPPolicies_test.csv",encoding = "utf-8"),delimiter = "\t")
test.head()

,id,question
0,0ce5c44a54d63a5b9b7c9657d1919fc7,天津市继续教育网在“抗疫知识专栏”中设有哪两类课程？
1,0e72999704f430d8aa5f1233d71bf0cb,北京市提供的优惠政策针对哪些重点企业？
2,0f17f97f38fb3174ab58acda1f1c3ff7,泉州市对举报违规餐饮店的举报人有何奖励？
3,c0c35634fdb83a40884fadebb0e82ff7,为保证疫情期间食品供应，广州市多少家商户上了饿了吗平台？
4,0caca9df37d5388f97ef351eb98cc58a,在北京市法律援助中心服务大厅开始对外提供法律咨询之前，如何获得法律咨询服务？


In [9]:
questiontest = test["question"]
questionstest = tokenization(questiontest)

In [10]:
tflist = []
for item in questionstest:
    scores = bm25Model.get_scores(item)
    dict = {}
    for (item1, item2) in zip(docid2context,scores):
        dict[item1] = item2
    tf_idfSorted = sorted(dict.items(),key=lambda x:x[1],reverse=True)
    counter = 0
    QkeywordSorted = []
    for item in tf_idfSorted:
        QkeywordSorted.append(item[0])
        counter+=1
        if counter == 20:
            break
    tflist.append(QkeywordSorted)

In [29]:
questionteststr = []
for item in questionstest:
    temp = ''
    for item1 in item:
        temp += ' '+item1
    questionteststr.append(temp)

In [73]:
questionteststr[0]

' 天津市 继续 教育网 抗疫 知识 专栏 设有 两类 课程'

In [11]:
id2query = {}
for item1,item2 in zip(test['id'],test['question']):
    id2query[item1] = item2

In [12]:
id2context = {}
for item1,item2 in zip(range(0,8932), docid2context):
    id2context[item1] = item2

In [48]:
with open('infer2.txt','a') as file_handle:
    for (item,item1) in zip(tflist,range(935,1643)):
        for item2 in item:
            file_handle.write(questionteststr[item1])
            file_handle.write('\t')
            file_handle.write(textstr[id2context[item2]])
            file_handle.write('\n')     

In [34]:
labelres[0:10]

[['1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0'],
 ['0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0'],
 ['0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0'],
 ['0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0'],
 ['0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0'],
 ['0',
  '0',

In [13]:
label = []
with open("result5.txt", "r") as f:
    for line in f.readlines():
        temp1,temp2,temp3 = line.strip().split('\t')
        label.append(temp3)
labelres = []
for item in range(0,len(label)//20):
    labelres.append(label[(item*20):(item*20+20)])

In [14]:
resid = []
for item,item1 in zip(tflist,labelres):
    temp = []
    for item2,item3 in zip(item,item1):
        if item3 == '1':
            temp.append(item2)
    resid.append(temp)

In [47]:
docid2context[resid[1][1]]

'助力中小微企业度难关 这些地方减租了 抗疫战斗仍在继续，北京市发文出台16条措施帮助中小微企业应对疫情影响，其中房租减免措施备受关注，中关村各空间载体将如何落实该项措施？ \u3000\u3000截至目前，中关村分园、特色产业园区纷纷推出减租措施，汇龙森、翠湖科创平台、中关村意谷等68家孵化器提出对房租进行减免，已推出的减免方案基本参照现行政策，减免租金15至30天。一起来看看它们的落实细则。 \u3000\u3000亦庄园：2月房租最高减免100% \u3000\u3000中小微企业承租区内国有企业房产从事经营活动，按照政府要求坚持营业或依照防疫规定关闭停业且不裁员、少裁员的，免收2月份房租；承租用于办公用房的，给予2月份租金50%的减免。鼓励产业园区、商务楼宇等单位为科技型中小微企业减免租金，具体由双方协商解决，给予一定资金补贴。 \u3000\u3000中关村创业大街：2月房租减免50% \u3000\u30002月6日，中关村创业大街出台房屋租金（含物业费）减免方案。对于受到疫情影响的科技型中小微企业和创业服务机构，中关村创业大街给予2月份租金50%的减免。此次减免惠及入驻街区的近百家机构及创业团队，缓解了疫情对企业造成的经营冲击。 \u3000\u3000汇龙森科技园：2月房租最高减免100% \u3000\u30001.在汇龙森产权范围内承租产业用房的中小微企业（需注册及纳税），给予2020年2月份房屋租金50%的减免。 \u3000\u30002.在防疫期间为国家防疫工作做出突出贡献的园区中小微企业，给予2020年2月份房屋租金100%的减免。 \u3000\u30003.在防疫期间为国家防疫工作做出突出贡献的园区中小微企业，暂时经营困难的，除给予2020年2月份房屋租金100%全免外，给予防疫後续期间全部房屋租金50%的减免。 \u3000\u3000锋创科技园：2月房租减免50% \u3000\u3000面对汹涌疫情，锋创科技园将对在园区内承租办公用房及商业的中小微企业（在北京经济技术开发区科创十三街18号院完成工商注册和属地纳税），给予2020年2月份租金50%减免。这是锋创科技园在疫情期间，继赠送消毒液、推行分时分批就餐、安装消毒帐篷、设立观察点等之後的又一举措，携手企业共抗疫情。 \u3000\u3000博奥共享平台：最高减免1个月房租 \

In [51]:
resid[0:100]

[['34830ac2ef5d314fbad41f7beed484f4',
  'd31c430fb2a83ca7a2545ced2bf16921',
  '5b7583bd20a83205b30b4b05db68234d'],
 ['fa91b4c9eb2132f4be0e936d6df9fa2f',
  '79f4770c7f1d34d497a0e15fe32fcbc9',
  '4777e4789a74371ea031d19280c32421',
  'c2d79264d2413dfeb989bc57c317cdb2',
  '09d2e7742e9631608f1ab1b9674d73a4'],
 ['b85afb2d621937799f35368f03da2f77',
  'e8bd012737af3325914fc255e70efd7c',
  '4f9000b974c53fe3a4fc3c90ba21252f',
  '4366d49083b639c08b69ae1b477488df',
  '132db7556ce23e7b9e5b1ed989690e8d',
  'e514f16e71c731e9a13d719e516cdad8',
  '5b83dc251754370bbc15ab798bdabdb2',
  'b998c24335a93dc8b9d6d41485321b4b',
  'a075e1bb4d2c37a5baa51825681a06f5',
  '00abdc76c4fc3ed6bcb2875ed7773b52',
  '344e7313125f34f1937842d480944993',
  '228bdf16f988362ba6caa88ee7c32052',
  'cf052836672a30cf8ba78694a82619f9',
  'fff94d4bf1fc3440b0465f2db91aab1d',
  '851e82a0c162350a88e73991ed004b7b',
  '7b8e23b123e531d490dc34ae3554d7b5',
  'afddc6683c903729885e574f591e01f9',
  '32f76f4f00853bf49487a3fb8b8387a0',
  'e9a7a0b

In [15]:
ts_text = []
ts_id = []
queryid = []
query = []
textset = []
for i,item2 in zip(range(0,1643,1),test['id']):
    if resid[i]:
        ts_id.append(resid[i][0])
        queryid.append(item2)
        query.append(id2query[item2])
        temp = []
        count = 0
        for item3 in resid[i]:
                temp.append(docid2context[item3])
                count +=1
                if count == 5:
                    break
        temp1 = set(temp)
        text = ""
        for item in temp:
            text +=item
        ts_text.append(text)
    else:
        ts_id.append(tflist[i][0])
        queryid.append(item2)
        query.append(id2query[item2])
        temp = []
        count = 0
        for item3 in tflist[i]:
                temp.append(docid2context[item3])
                count +=1
                if count == 5:
                    break
        temp1 = set(temp)
        text = ""
        for item in temp:
            text +=item
        ts_text.append(text)

testcsv = pd.DataFrame(columns = ('id','docid','question','text'))
# test["docid"] = ts_id
#test["text"] = ts_text

testcsv['docid'] = ts_id
testcsv['id'] = queryid
testcsv['question'] = query
testcsv['text'] = textset

testcsv.head()

testcsv.to_csv('test_recall.csv',index = 0)

In [70]:
len(ts_text)

1643

In [16]:
test['docid'] = ts_id
test["answer"] = ["1"]*1643
test['text'] = ts_text

In [17]:
test

,id,question,docid,answer,text
0,0ce5c44a54d63a5b9b7c9657d1919fc7,天津市继续教育网在“抗疫知识专栏”中设有哪两类课程？,34830ac2ef5d314fbad41f7beed484f4,1,市人社局关于在天津市专业技术人才继续教育网增设抗疫知识专栏的通知 市人社局关于在天津市专业技...
1,0e72999704f430d8aa5f1233d71bf0cb,北京市提供的优惠政策针对哪些重点企业？,fa91b4c9eb2132f4be0e936d6df9fa2f,1,助力中小微企业度难关 这些地方减租了 抗疫战斗仍在继续，北京市发文出台16条措施帮助中小微企...
2,0f17f97f38fb3174ab58acda1f1c3ff7,泉州市对举报违规餐饮店的举报人有何奖励？,b85afb2d621937799f35368f03da2f77,1,南安：举报违规餐饮店 奖励口罩50个 昨日，《南安市防控新型冠状病毒感染的肺炎疫情应急指挥部...
3,c0c35634fdb83a40884fadebb0e82ff7,为保证疫情期间食品供应，广州市多少家商户上了饿了吗平台？,6ddc478f47b1313987d9b988b0be2a24,1,民生物资“不愁买” 直播电商“定民心” 在新型冠状病毒感染的肺炎疫情防控工作中，广州市商务局...
4,0caca9df37d5388f97ef351eb98cc58a,在北京市法律援助中心服务大厅开始对外提供法律咨询之前，如何获得法律咨询服务？,7c2f04c841cb3afa8dcbeeff18c242be,1,市法援中心2月3日起对外提供法律咨询 北京市法律援助中心服务大厅将于2020年2月3日（农历...
...,...,...,...,...,...
1638,53e0719047fc3326963a10a48bd924d4,湘潭市人民政府对中小企业的社会保险费采取什么样的政策？,7d044d5693703c2f8c7773f0a48f4c79,1,湘潭市人民政府关于应对新型冠状病毒感染的肺炎疫情支持中小企业平稳健康发展的若干意见 各县市区...
1639,a27e530350ed33438a34f675f0fc461a,龙岩市人社和医疗保障业务网上业务申报系统用户登录方式是什么？,ed4c44aff7f130439c283a7777aee0d7,1,龙岩市人力资源和社会保障局关于推广龙岩市人社和医疗保障公共服务网相关工作的通知 各县(市、区...
1640,e4e75261297f3a49b0b7679920ba68a8,国家对受疫情影响较大的困难行业企业2020年度发生的亏损如何处理？,aec738bbd7a730ef9b0f06ac32b09cce,1,重庆对受疫情影响较大的困难行业企业亏损最长结转年限延长至8年 国家税务总局重庆市税务局党...
1641,345641e320da302b8b5e39cdb2b003a9,税务总局关于支持新型冠状病毒感染的肺炎疫情防控有关个人所得税政策的公告什么时间起实施？,52bde519e70a3029a3cb09b764c08bf2,1,财政部 税务总局关于支持新型冠状病毒感染的肺炎疫情防控有关个人所得税政策的公告 财政部 税务...


In [18]:
def data_json(data):
    res = {}
    count=0
    data_dic = []
    for i in range(len(data)):
        
        question = " ".join(data.iloc[i,2])
        text = " ".join("".join(data.iloc[i,4].split()))
        answers = " ".join("".join(data.iloc[i,3].split()))
        answers_start = text.find(answers)
        if answers_start==-1:
            count+=1
            print(i)
        dic = {}
        dic["context"] = text
        
        dic["qas"] = [{"answers":[{"answer_start":answers_start,"text":answers}],"question":question,"id":data.iloc[i,0]}]
        data_dic.append({"title":"疫情","paragraphs":[dic]})
                      
    res["data"] = data_dic
    print("么有答案的样本{}".format(count))
    return res

In [19]:
import json

In [20]:
test_doc = test[["id","docid","question","answer","text"]]
test_json = data_json(test_doc)
with open(r'C:\Users\40821\Desktop\knowledgegraph\epidemic-QA-assistant--2020-master\test_torch_0430_N3_json.json','w',encoding = 'utf-8') as f:
    json.dump(test_json,f,ensure_ascii=False)
f.close()

88
89
147
264
298
439
762
847
868
1090
1350
1404
1528
1542
么有答案的样本14


In [21]:

import json
test_ = json.load(open(r"C:\Users\40821\Desktop\knowledgegraph\epidemic-QA-assistant--2020-master\predictions_0430.json",encoding = "utf-8"))
test_dic = {}
for i in test_.keys():
    test_dic[i] = "".join(test_[i].split())    
test_doc["answer"] = test_dic.values()
test_doc[["id","docid","answer"]].to_csv(r"C:\Users\40821\Desktop\knowledgegraph\epidemic-QA-assistant--2020-master\test_predictions_0430.csv",encoding = "utf-8",sep = "\t",index = None)